# Module 7 - Programming Assignment

## Directions

There are general instructions on Blackboard and in the Syllabus for Programming Assignments. This Notebook also has instructions specific to this assignment. Read all the instructions carefully and make sure you understand them. Please ask questions on the discussion boards or email me at `EN605.445@gmail.com` if you do not understand something.

<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
You must follow the directions *exactly* or you will get a 0 on the assignment.
</div>

You must submit a zip file of your assignment and associated files (if there are any) to Blackboard. The zip file will be named after you JHED ID: `<jhed_id>.zip`. It will not include any other information. Inside this zip file should be the following directory structure:

```
<jhed_id>
    |
    +--module-01-programming.ipynb
    +--module-01-programming.html
    +--(any other files)
```

For example, do not name  your directory `programming_assignment_01` and do not name your directory `smith122_pr1` or any else. It must be only your JHED ID.

Imports here if needed.

In [1]:
import copy

# Forward Planner

## Unify

Use the accompanying `unification.py` file for unification. For this assignment, you're almost certainly going to want to be able to:

1. specify the problem in terms of S-expressions.
2. parse them.
3. work with the parsed versions.

`parse` and `unification` work exactly like the programming assignment for last time.

In [2]:
from unification import parse, unification

## Forward Planner

In this assigment, you're going to implement a Forward Planner. What does that mean? If you look in your book, you will not find pseudocode for a forward planner. It just says "use state space search" but this is less than helpful and it's a bit more complicated than that. **(but please please do not try to implement STRIPS or GraphPlan...that is wrong).**

At a high level, a forward planner takes the current state of the world $S_0$ and attempts to derive a plan, basically by Depth First Search. We have all the ingredients we said we would need in Module 1: states, actions, a transition function and a goal test. We have a set of predicates that describe a state (and therefore all possible states), we have actions and we have, at least, an implicit transition function: applying an action in a state causes the state to change as described by the add and delete lists.

Let's say we have a drill that's an item, two places such as home and store, and we know that I'm at home and the drill is at the store and I want to go buy a drill (have it be at home). We might represent that as:

<code>
start_state = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Store)"
]
</code>

And we have a goal state:

<code>
goal = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)"
]
</code>

The actions/operators are:

<code>
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}
</code>

These will all need to be parsed from s-expressions to the underlying Python representation before you can use them. You might as well do it at the start of your algorithm, once. The order of the conditions is *not* arbitrary. It is much, much better for the unification and backtracking if you have the "type" predicates (item, place, agent) before the more complex ones. Trust me on this.

As for the algorithm itself, there is going to be an *outer* level of search and an *inner* level of search.

The *outer* level of search that is exactly what I describe here: you have a state, you generate successor states by applying actions to the current state, you examine those successor states as we did at the first week of the semester and if one is the goal you stop, if you see a repeat state, you put it on the explored list (you should implement graph search not tree search). What could be simpler?

It turns out the Devil is in the details. There is an *inner* level of search hidden in "you generate successor states by applying actions to the current state". Where?

How do you know if an action applies in a state? Only if the preconditions successfully unify with the current state. That seems easy enough...you check each predicate in the conditions to see if it unifies with the current state and if it does, you use the substitution list on the action, the add and delete lists and create the successor state based on them.

Except for one small problem...there may be more than one way to unify an action with the current state. You must essentially search for all successful unifications of the candidate action and the current state. This is where my question through the semester appliesm, "how would you modify state space search to return all the paths to the goal?"

Unification can be seen as state space search by trying to unify the first precondition with the current state, progressively working your way through the precondition list. If you fail at any point, you may need to backtrack because there might have been another unification of that predicate that would succeed. Similarly, as already mentioned, there may be more than one.

So...by using unification and a properly defined <code>successors</code> function, you should be able to apply graph based search to the problem and return a "path" through the states from the initial state to the goal. You'll definitely want to use graph-based search since <code>( drive Me Store), (drive Me Home), (drive Me Store), (drive Me Home), (drive Me Store), (buy Me Store Drill), (drive Me Home)</code> is a valid plan.

Your function should return the plan...but if you pass an extra debug=True parameter, it should also return the intermediate *states* as well as the actions.

-----

# James Duvall

# July 16, 2017

**parse_state(state)**

A simple function that is used to take either the starting state or goal state of s-expressions and parse them into lists of lists using the parse() function.

In [3]:
def parse_state(state):
    state_list = []
    for expression in state:
        state_list.append(parse(expression))
    return state_list

**parse_actions(actions)**

Another simple function that is used to take in the given actions of s-expressions and parse them into the list-of-lists format using the parse() function.

In [4]:
def parse_actions(actions):
    actions_copy = copy.deepcopy(actions)
    for action in actions_copy:
        for item in actions_copy[action]:
            if isinstance(actions_copy[action][item], list):
                parsed_list = []
                for values in actions_copy[action][item]:
                    parsed_list.append(parse(values))
                actions_copy[action][item] = parsed_list
            else:
                actions_copy[action][item] = parse(actions_copy[action][item])
    return actions_copy

**pretty_print_plan(plan, debug)**

Another simple function. This takes as both the plan and desired debugging information (True or False) and prints each new action or state/action pair, line-by-line.

In [5]:
def pretty_print_plan(plan, debug):
    for item in plan:
        if debug == True:
            print "Action: ", item[1]
            print "State: ", item[0]
            print ""
        else:
            print "Action: ", item
            print ""

**inner_successors(preconditions, state)**

The first of my main functions, here I am writing them in order from the inside state-space search to the outer level of state-space search.

This is the successors function for the inner level of search. It takes two inputs:

(1) preconditions - the list of preconditions for a given action

(2) state - the current state

And has one output:

(1) result - the list of children to check in the inner_search() function

This maintains three important pieces of unformation for each state. It holds the preconditions as the first element in a list, a dictionary of unified actions as the second element in the list, and lastly a counter which is used to check whether we were able to unify with each precondition. Note this counter is a number between 0 and n, where n is the number of preconditions for a given action. 

This function enumerates through each of the conditionds of the current state and attempts to unify each one with a given precondition inficated by the counter. So essentially, we are looking at one precondition at a time and seeing whether that unifies with any of the conditions in the state. If it does unify, we update the unification dictionary and the counter in to move onto the next precondition. Lastly, we loop through the preconditions in the action and replace them with the successfully unified condition. We then return the result as a list.

In [6]:
def inner_successors(preconditions, state):
    result = []
    for index, item in enumerate(state):
        unified = unification(item, preconditions[0][preconditions[2]])
        if unified != False:
            precond_update = copy.deepcopy(preconditions)
            precond_update[1].update(unified) # update dict with new info
            precond_update[2]+=1 # update the precondition we are looking at
            for index1, sublist in enumerate(precond_update[0]):
                for index2, item in enumerate(sublist):
                    if item in unified.keys():
                        precond_update[0][index1][index2] = unified[item]
            result.append(precond_update)
    return result

**inner_search(preconditions, state)**

This is the inner level of search and takes two inputs:

(1) preconditions - the preconditions of a given action to check

(2) state - the current state

And returns:

(1) unifiers - a list of dictionaries containing all of the successful unifications for the preconditions of a given action

This inner search follows the main graph search algorithm, however, it has been modified that if we find a successful unification, we add this to the "unifiers" list and continue progressing, backtracking as needed. So instead of returning the explored list containing everything visited up until the goal, we return the unifiers list containing all possible ways to reach the goal. 

For a given action, we assign the preconditions (with no assignments) to the frontier (along with the unifiers and the counter). If the counter has incremented to be 1 more than the number of preconditions, then it must be a successful unification! And so we add it to the unifiers result list. Otherwise, we call inner_successors() to find the unifying children for the current state. The rest is from the algorithm.

In [7]:
def inner_search(preconditions, state):
    unifiers = []
    frontier = [[copy.deepcopy(preconditions),{},0]]
    explored = []
    while frontier != []:
        current = frontier.pop(-1)
        if current[2] == len(current[0]):
            unifiers.append(current[1])
            explored.append(current)
            continue
        inner_children = inner_successors(current,state)
        for child in inner_children:
            if (child not in frontier) and (child not in explored):
                frontier.append(child)
        explored.append(current)
    return unifiers

**successors(state, actions)**

The successors function for the outer level of search.

The inputs are:

(1) state - the current state on which to generate a new state

(2) actions - the set of actions we are concerned with

The output is:

(1) transition_states - a list containing both the new state and the action taken for that state

This function loops through all possible actions. It then calls inner_search() to generate all possible unifying assignments for that action. For each of these possible unifiers, we calculate the action taken and modify the state based on the add and delete lists. I took a shortcut here. Since the add and delete lists are each one item for both actions, I simply find and replace the one to delete. This way, when checking whether a successor state (in forward_plan()) exists, I know it will be in the same order as one that already exists in the frontier and explored lists. I then return the list of possible new states (and actions).

In [8]:
def successors(state, actions):
    transition_states = []
    
    for action in actions:
        unifiers = inner_search(actions[action]["conditions"], state)

        for results in unifiers:
            updated_state = copy.deepcopy(state)
            
            state_action = copy.deepcopy(actions[action]["action"])
            for index, condition in enumerate(state_action):
                if condition in results.keys():
                    state_action[index] = results.get(condition)
            
            action_delete = copy.deepcopy(actions[action]["delete"][0])
            for index, condition in enumerate(action_delete):
                if condition in results.keys():
                    action_delete[index] = results.get(condition)
                    
            action_add = copy.deepcopy(actions[action]["add"][0])
            for index, condition in enumerate(action_add):
                if condition in results.keys():
                    action_add[index] = results.get(condition)

            updated_state[updated_state.index(action_delete)] = action_add            
            
            transition_states.append([updated_state, state_action])
            
    return transition_states

**path(list_of_states, debug)**

The last of the simple functions. This takes the list of states required to reach the goal and the debug option of True or False. If debug is false, the path simply represents the actions taken to reach the goal. If debug is true, the path also includes the state represented from the result of that path.

In [9]:
def path(list_of_states, debug):
    plan = []
    if debug == True:
        for item in list_of_states:
            plan.append(item)
        plan.append([list_of_states[-1][0],["Success!"]])
        return plan
    else:
        for item in list_of_states:
            plan.append(item[1])
        plan.append(["Success!"])
        return plan

**forward_planner(start_state, goal, actions, debug)**

The main algorithm. It takes the following inputs:

(1) start_state - the starting state of the problem

(2) goal - the goal state we want to reach

(3) actions - the actions we can take to help us reach the goal

(4) debug - the true/false boolean that tells us what to print.

Returned is a "pretty version" of the final plan printed to the console

The function first initially parses the starting state, the goal state, and the actions from s-expressions into appropriate lists. We then follow the main graph-based algorithm for state-space search (note I moved the "explored.append(current)" from line 9 to line 5 in the pseudocode based on our email exchange about the edge case. All the work is pretty much done in the previously described functions.

If we don't find a plan, return None.



In [10]:
def forward_planner(start_state, goal, actions, debug=False):
    
    start = parse_state(start_state)
    final_goal = parse_state(goal)
    actions_parsed = parse_actions(actions)
    
    frontier = copy.deepcopy([[start, ["Start."]]])
    explored = []
    while frontier != []:
        current = frontier.pop(-1)
        explored.append(current)
        if final_goal == current[0]:
            return pretty_print_plan(path(explored, debug), debug)
        children = successors(current[0], actions_parsed)
        for child in children:
            if child[0] not in [item[0] for item in frontier] and child[0] not in [item[0] for item in explored]:
                frontier.append(child)
    return None

You will be solving the problem from above. Here is the start state:

In [11]:
start_state = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Store)"
]

The goal state:

In [12]:
goal = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)"
]

and the actions/operators:

In [13]:
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

This is the result of the forward planner without using the debug option.

In [14]:
forward_planner(start_state, goal, actions)

Action:  ['Start.']

Action:  ['drive', 'Me', 'Home', 'Store']

Action:  ['buy', 'Me', 'Store', 'Drill']

Action:  ['drive', 'Me', 'Store', 'Home']

Action:  ['Success!']



This is the result of the forward planner using the debug option to also show the intermediate states of the problem.

In [15]:
forward_planner(start_state, goal, actions, debug=True)

Action:  ['Start.']
State:  [['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['agent', 'Me'], ['at', 'Me', 'Home'], ['at', 'Drill', 'Store']]

Action:  ['drive', 'Me', 'Home', 'Store']
State:  [['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['agent', 'Me'], ['at', 'Me', 'Store'], ['at', 'Drill', 'Store']]

Action:  ['buy', 'Me', 'Store', 'Drill']
State:  [['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['agent', 'Me'], ['at', 'Me', 'Store'], ['at', 'Drill', 'Me']]

Action:  ['drive', 'Me', 'Store', 'Home']
State:  [['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['agent', 'Me'], ['at', 'Me', 'Home'], ['at', 'Drill', 'Me']]

Action:  ['Success!']
State:  [['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['agent', 'Me'], ['at', 'Me', 'Home'], ['at', 'Drill', 'Me']]

